# A Kumon maths scoring system

My kids study Kumon - a paid English and Maths tutoring system with papers. They wanted trophies and to get competitive about it. So I created a scoring system for them.
In code - they can see and we can discuss the rules.


In [12]:
import datetime
import yaml

We have the start and end date.
We should next collect the raw data for the papers for each child.

In [13]:
grade_scores = {
    100: 6,
    "A": 4,
    "B": 3,
    "C": 2,
    "D": 1
}

In [14]:
def score_paper(paper, explain=False, jackpot=100):
    # Are the corrections done
    if explain:
        print(f"Scoring paper for {paper['date']}")
    # convert and sum the scores
    score_values = [grade_scores.get(grade, 0) * 2 for grade in paper["grades"]]
    score = sum(score_values)
    if explain:
        print(f"\tGrades alone give {score} points.")
    # add 100% bonus - 6 additional points
    if all(grade==jackpot for grade in paper["grades"]):
        score += 6
        if explain:
            print(f"\tBonus! 100% - adding 6 points.  Total is {score}")
    # Now score the duration - 20 minutes is optimal.
    max_duration = min(paper['duration'], 40)
    score += int(max_duration / 10)
    if explain:
        print(f"\tAdding a time bonus. Total is {score}")

    if max_duration > 15 and max_duration < 25:
        score += 4
        if explain:
            print(f"\tClose to 20! Bonus of 4 points! Total is {score}")


    # add before time bonus
    if paper["completed_in_time"]:
        score *= 1.5
        if explain:
            print(f"\tBonus! Completed in time - 1.5 times score! total is {score}")

    return score


In [15]:
def score_child(child):
    paper_scores = [score_paper(paper) for paper in child["papers"] if paper['corrections_completed']]
    return sum(paper_scores)

In [16]:
def summarize_scores(children):
    highest_score = (None, 0)
    for child_name in children.keys():
        score = score_child(children[child_name])
        print(f"{child_name}: {score}")
        if score > highest_score[1]:
            highest_score = (child_name, score)
    print(f"Current lead goes to {highest_score[0]}")


In time is defined as: 
* Before 3pm on non-school days
* Before 6pm on school days

In [22]:
with open("maths_data.yaml") as children_fd:
    try:
        children = yaml.safe_load(children_fd)['children']
    except yaml.parser.ParserError as e:
        print(e)

In [23]:
score_paper(children["Jonathan"]["papers"][1], explain=True)

Scoring paper for 2021-08-24
	Grades alone give 18 points.
	Adding a time bonus. Total is 19
	Bonus! Completed in time - 1.5 times score! total is 28.5


28.5

In [24]:
score_child(children['Helena']), score_child(children['Jonathan'])

(0, 40.5)

In [25]:
summarize_scores(children)

Helena: 0
Jonathan: 40.5
Current lead goes to Jonathan
